In [1]:
#Importing Necessary Packages
import requests
import os
import urllib3
import itertools 
import re
import urllib.request
import pandas as pd
import numpy as np
os.chdir("C:\\Users\\Rahul\\Desktop\\Python Programs\\Machine Learning\\DataSets\\Used Cars v3")

In [2]:
## From carDekho Main Page fetching html source code

## Fetching Main File HTML code
response =urllib.request.urlopen('https://www.cardekho.com/newcars')
html = response.read()
text = html.decode()

##Fetching All Brand Names from HTML data and saving all as brands list
brands=re.findall('class="BrIconNewCar" href="(/cars/(.*?))">',text)
brands=list(itertools.chain(*brands))

## Converting brands names in website links by concatenating cardekho.com for further data scrapping
link_list=[]
count=0
for i in brands:
    if count%2==0:
        link_list.append('https://www.cardekho.com'+i)
    else:
        link_list.append(i) 
    count+=1
link_list   

In [5]:
# Fetching Brands of All Models via link list created above one by one
count1=0
link_model=[]
for i in link_list:
    k=i.lower()                                   # to lower brand name for site compatible
    
# Traversing model links using brand name and adding in link_model list
# if count divisible by zero then open link and then in next iteration fetching model and concating link in model_list

    if count1%2==0:
        count1+=1
        response =urllib.request.urlopen(i)
        html = response.read()
        text = html.decode()
    else:
        count1+=1
       # regex_m1=re.compile(r'href="/'+k+'/(.*?)">|href="/carmodels/'+i+'/(.*?)">')
    
#Using regex for pattern matching of models as per html souce code recd from above reuqest
        regex_m1=re.compile('href="/'+k+'/(.*?)">')
        regex_m2=re.compile('href="/carmodels/'+i+'/(.*?)">')
        model1=re.findall(regex_m1,text)
        model2=re.findall(regex_m2,text)
        for j in model1:
            link_model.append('https://www.cardekho.com/carmodels/'+i+'/'+j) 
        for j in model2:
            link_model.append('https://www.cardekho.com/carmodels/'+i+'/'+j)       

link_model

['https://www.cardekho.com/carmodels/Maruti/swift',
 'https://www.cardekho.com/carmodels/Maruti/swift-dzire',
 'https://www.cardekho.com/carmodels/Maruti/alto-k10',
 'https://www.cardekho.com/carmodels/Maruti/xl6',
 'https://www.cardekho.com/carmodels/Maruti/celerio-x',
 'https://www.cardekho.com/carmodels/Maruti/baleno-rs',
 'https://www.cardekho.com/carmodels/Maruti/future-s',
 'https://www.cardekho.com/carmodels/Maruti/wagonr-electric',
 'https://www.cardekho.com/carmodels/Maruti/jimny',
 'https://www.cardekho.com/carmodels/Maruti/Maruti_Baleno',
 'https://www.cardekho.com/carmodels/Maruti/Maruti_Vitara_Brezza',
 'https://www.cardekho.com/carmodels/Maruti/Maruti_Wagon_R',
 'https://www.cardekho.com/carmodels/Maruti/Maruti_Ertiga',
 'https://www.cardekho.com/carmodels/Maruti/Maruti_Alto_800',
 'https://www.cardekho.com/carmodels/Maruti/Maruti_Celerio',
 'https://www.cardekho.com/carmodels/Maruti/Maruti_Ciaz',
 'https://www.cardekho.com/carmodels/Maruti/Maruti_SX4_S_Cross',
 'https://

In [25]:
brand_all=[]        #List all brands name
model_all=[]        #List to store name of the product
variant_all=[]      #List all variants name
price_all=[]        #List to store price of the product

for i in link_model:
    url = i
    
#Seprating brand and model names from the link_list for easy analysing
    brand_index=i.rfind('/', 0, i.rfind('/'))
    model_index=i.rfind('/')
    brand=i[brand_index+1:model_index]
    model=i[model_index+1:]
    if model.find(brand,0,len(brand))>=0:
        model=model[len(brand)+1:]

#Gettig price of all cars from link_list and saving in price_all list
    try:
        response =urllib.request.urlopen(url)
    except:
        continue
    html = response.read()
    text = html.decode()
    
#Using regex function to search pattern of rpice of cars
    regex_v1=re.compile('data-variant="(.*?)"><td>')
        
    regex_p1=re.compile('<i class="icon-cd_R">Rs.</i>([0-9\.]+) Lakh<sup>\*</sup><div class="orpbtn')
    regex_p2=re.compile('<i class="icon-cd_R">Rs.</i>([0-9\.]+) Cr<sup>\*</sup><div class="orpbtn')

  
    price1=re.findall(regex_p1,text)
    price2=re.findall(regex_p2,text) 
    variant=re.findall(regex_v1,text)

#Once found price adding brand,model,variant and price in respective list,so index and length of all are same.
    for x in range(0,len(variant)):
        
        brand_all.append(brand)
        model_all.append(model)
        variant_all.append(variant[x])
        
#Checking whether price in Lakh Or Cr and appenidng it accrodingly and if price missing the filling by nan   
        try:
            if price1[x]:
                price_all.append(price1[x]+" Lakh")
            elif price2:
                price_all.append(price2[x]+" Cr")
            else:
                price_all.append('Nan')
        except:
            if price2:
                price_all.append(price2[x-len(price1)]+" Cr")
            else:
                price_all.append('Nan')


#print(brand_all)
#print(model_all)
#print(variant_all)
#print(price_all)

In [27]:
print(len(brand_all))
print(len(model_all))
print(len(variant_all))
print(len(price_all))

2245
2245
2245
2245


In [28]:
#creating a dataframe and appending brands,model,price and variant in cars and then saving as excel file
cars=pd.DataFrame()
cars['Brand']=np.array(brand_all)
cars['Model']=np.array(model_all)
cars['Price']=np.array(price_all)
cars['Variant']=np.array(variant_all)
cars.to_excel("cars_scrapped_data.xlsx")